# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

In [17]:
import pandas as pd

with open ('female.txt', encoding = 'utf-8') as f:
    female = set(map(str.strip, f.readlines()))
    #female = f.read()
with open ('male.txt', encoding = 'utf-8') as f:
    male = set(map(str.strip, f.readlines()))
    #male = f.read()

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

    Неоднозначные имена - те, которые одновременно и мужские и женские

In [40]:
ambigous =  male.intersection(female)

In [21]:
male -= ambigous
female -= ambigous
display(len(male))
display(len(female))

2578

4635

После удаления неоднозначных имен - количество мужских и женских имен - 2578 и 4635 соответственно. Для балланса и ровного счета возьмём по 2500 имён и разобьём на тестовую и обучающие выборки по 1900 и 600 имён.

In [46]:
import itertools
DataFemale = pd.DataFrame(
        sorted(itertools.chain(
            zip(female, itertools.repeat('F'))
        )),
        columns=['name', 'gender'],
    ).set_index('name')
DataFemale['gender'] = DataFemale.gender.astype('category')

In [47]:
DataMale = pd.DataFrame(
        sorted(itertools.chain(
            zip(male, itertools.repeat('M'))
        )),
        columns=['name', 'gender'],
    ).set_index('name')
DataMale['gender'] = DataMale.gender.astype('category')

In [50]:
DataFemale = DataFemale[:2500].copy()
DataMale = DataMale[:2500].copy()

In [51]:
DFemale_train = DataFemale.iloc[:1900, :]
DFemale_test = DataFemale.iloc[1900:, :]
DMale_train = DataMale.iloc[:1900, :]
DMale_test = DataMale.iloc[1900:, :]

In [55]:
DF_train = pd.concat([DFemale_train, DMale_train])
DF_test = pd.concat([DFemale_test, DMale_test])

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [60]:
from nltk.util import ngrams
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline

In [146]:
def predic_bayes(train,test,Val1,Val2):
        clf = Pipeline([
            ('vect',CountVectorizer(analyzer='char', ngram_range=(Val1,Val2))),
            ('clf',  MultinomialNB()),
        ])
        clf.fit(train.index, train.gender)   
        predict = clf.predict(test.index)
        print("ngram_range=({0:1.0f},{1:2.0f}) Accuracy:{2:4.2f} F1-measure:{3:4.2f} Precision:{4:.2f} Recall:{5:.2f}".format(
            Val1, 
            Val2, 
            accuracy_score(test.gender, predict),
            f1_score(test.gender, predict, average='macro'),
            precision_score(test.gender, predict, average='macro'),
            recall_score(test.gender, predict, average='macro')
        ))    

In [147]:
for i in range(6):
    for j in range(6):
        v1 = i + 1
        v2 = j + 1
        try:
            predic_bayes(DF_train,DF_test,v1,v2)
        except:
            error = True        

ngram_range=(1, 1) Accuracy:0.48 F1-measure:0.44 Precision:0.47 Recall:0.48
ngram_range=(1, 2) Accuracy:0.54 F1-measure:0.52 Precision:0.55 Recall:0.54
ngram_range=(1, 3) Accuracy:0.60 F1-measure:0.60 Precision:0.61 Recall:0.60
ngram_range=(1, 4) Accuracy:0.63 F1-measure:0.63 Precision:0.64 Recall:0.63
ngram_range=(1, 5) Accuracy:0.63 F1-measure:0.63 Precision:0.64 Recall:0.63
ngram_range=(1, 6) Accuracy:0.63 F1-measure:0.63 Precision:0.64 Recall:0.63
ngram_range=(2, 2) Accuracy:0.60 F1-measure:0.60 Precision:0.61 Recall:0.60
ngram_range=(2, 3) Accuracy:0.65 F1-measure:0.64 Precision:0.65 Recall:0.65
ngram_range=(2, 4) Accuracy:0.68 F1-measure:0.67 Precision:0.68 Recall:0.68
ngram_range=(2, 5) Accuracy:0.68 F1-measure:0.68 Precision:0.68 Recall:0.68
ngram_range=(2, 6) Accuracy:0.67 F1-measure:0.67 Precision:0.68 Recall:0.67
ngram_range=(3, 3) Accuracy:0.65 F1-measure:0.65 Precision:0.65 Recall:0.65
ngram_range=(3, 4) Accuracy:0.67 F1-measure:0.67 Precision:0.67 Recall:0.67
ngram_range=

Лучший результат:

In [148]:
predic_bayes(DF_train,DF_test,2,5)

ngram_range=(2, 5) Accuracy:0.68 F1-measure:0.68 Precision:0.68 Recall:0.68


In [174]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def predic_logistic(train,test,n):
    clf = Pipeline([
        ('vect', CountVectorizer(lowercase=False, analyzer='char', ngram_range=(n, n))),
        ('normalizer', StandardScaler(with_mean=False)),
        ('clf', LogisticRegression(random_state=234))
    ])        
    clf.fit(train.index, train.gender)   
    predict = clf.predict(test.index)
    print("n={0:1.0f} Accuracy:{1:4.2f} F1-measure:{2:4.2f} Precision:{3:.2f} Recall:{4:.2f}".format(
            n, 
            accuracy_score(test.gender, predict),
            f1_score(test.gender, predict, average='macro'),
            precision_score(test.gender, predict, average='macro'),
            recall_score(test.gender, predict, average='macro')
        ))   

In [176]:
for i in range(3):
    v1 = i + 2
    try:
        predic_logistic(DF_train,DF_test,v1)
    except:
        error = True   

/home/den/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/den/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


n=2 Accuracy:0.46 F1-measure:0.40 Precision:0.43 Recall:0.46
n=3 Accuracy:0.64 F1-measure:0.64 Precision:0.65 Recall:0.64
n=4 Accuracy:0.67 F1-measure:0.65 Precision:0.69 Recall:0.67


/home/den/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Лучший результат, n=4:

In [179]:
predic_logistic(DF_train,DF_test,4)

n=4 Accuracy:0.67 F1-measure:0.65 Precision:0.69 Recall:0.67


/home/den/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


##  Часть 3. Нейронная сеть


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor